# Solution

In [76]:
import pandas as pd

# Display DataFrame without wrapping the cols
pd.set_option('display.max_columns', None) # Set to None to display all columns
pd.set_option('display.width', 1000) # Set width to a high value to prevent wrapping

# Loading csv files into DFs
# df_cdhdr = pd.read_csv('datasets/CDHDR.csv')
df_cdpos = pd.read_csv('datasets/CDPOS.csv', dtype='object')
# df_ekko = pd.read_csv('datasets/EKKO.csv')
df_ekpo = pd.read_csv('datasets/EKPO.csv', dtype='object')

In [77]:
df_ekpo['NETWR'] = df_ekpo['NETWR'].astype(float)

### Notes:

- It seems that the only table needed to complete this task is CDPOS, because:
    - The goal is to sum the NETWR of all items of a PO on the date entered by the user
        - I will assume for the task that the date can be either day/month/year and we will have to aggregate
    - The 1st obstacle informs that simply summing all PO_Items for a specific PO from EKPO table is not correct (otherwise table 'group_items_ekpo' below this markdown would do the job.)
    - The 2nd obstacle informs that for the actual price of the order (on a given date) we need to use CDPOS, not EKPO
    - CDPOS contains basically the same info as EKPO, but more 

In [78]:
# If the date condition would not be provided we could simply group by EBELN and sum all NETWR
group_items_ekpo = df_ekpo.groupby('EBELN').agg({'NETWR':'sum'})
group_items_ekpo

,NETWR
EBELN,
0071154204,65839.52
0071154509,34000.00
0071154665,6792.91
0071155226,28401.60
0071155605,2518.01
...,...
4700001512,90000.00
4700001545,89000.00
4700001551,16000.00


The example below illustrates that on 2021-08-23 value for item with TABKEY 10007210940900030 was changed from 39750.00 to 48750.00 just after 2 mins (probably an error). So it makes sense to aggregate to a day (but hour, min or sec would not really introduce any important insight).

In [79]:
filter_2 = df_cdpos[(df_cdpos['FNAME'] == 'NETWR') & (df_cdpos['TABKEY'] == '010007210940900030')]

filter_2.sort_values('UDATE').head(2)

,MANDANT,OBJECTCLAS,OBJECTID,CHANGENR,TABNAME,TABKEY,FNAME,CHNGIND,VALUE_NEW,VALUE_OLD,UDATE,UTIME
1903,010,EINKBELEG,0072109409,0071305379,EKPO,010007210940900030,NETWR,U,39750.00,30750.00,2021-08-23T00:00:00,1970-01-01T13:01:32
1907,010,EINKBELEG,0072109409,0071305388,EKPO,010007210940900030,NETWR,U,48750.00,39750.00,2021-08-23T00:00:00,1970-01-01T13:03:45


We need to convert 'VALUE_NEW' and 'VALUE_OLD' to float but running the code below returned 'ValueError: could not convert string to float: 'Turnus und Stichprobe für 2018'

So we will have to clean up these two columns a bit

In [80]:
# df_cdpos['VALUE_NEW'] = df_cdpos['VALUE_NEW'].astype(float) #returned error (contains non-numerical strings)
# df_cdpos['VALUE_OLD'] = df_cdpos['VALUE_OLD'].astype(float) #returned error (contains non-numerical strings)

In [81]:
df_cdpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   MANDANT     2478 non-null   object
 1   OBJECTCLAS  2478 non-null   object
 2   OBJECTID    2478 non-null   object
 3   CHANGENR    2478 non-null   object
 4   TABNAME     2478 non-null   object
 5   TABKEY      2478 non-null   object
 6   FNAME       2478 non-null   object
 7   CHNGIND     2478 non-null   object
 8   VALUE_NEW   2399 non-null   object
 9   VALUE_OLD   2264 non-null   object
 10  UDATE       2478 non-null   object
 11  UTIME       2478 non-null   object
dtypes: object(12)
memory usage: 232.4+ KB


In [82]:
df_cdpos.isna().sum()

MANDANT         0
OBJECTCLAS      0
OBJECTID        0
CHANGENR        0
TABNAME         0
TABKEY          0
FNAME           0
CHNGIND         0
VALUE_NEW      79
VALUE_OLD     214
UDATE           0
UTIME           0
dtype: int64

In [83]:
nan_values = df_cdpos[df_cdpos['VALUE_NEW'].isna()]

# Print the DataFrame containing NaN values in 'VALUE_NEW' column
print(nan_values)

     MANDANT OBJECTCLAS    OBJECTID    CHANGENR TABNAME              TABKEY  FNAME CHNGIND VALUE_NEW VALUE_OLD                UDATE                UTIME
18       010  EINKBELEG  0072097616  0051998659    EKPO  010007209761600020    KEY       I       NaN       NaN  2018-01-18T00:00:00  1970-01-01T15:17:28
85       010  EINKBELEG  0072097969  0052127853    EKPO  010007209796900020    KEY       I       NaN       NaN  2018-02-05T00:00:00  1970-01-01T10:46:51
86       010  EINKBELEG  0072097969  0052128438    EKPO  010007209796900020  TWRKZ       U       NaN         2  2018-02-05T00:00:00  1970-01-01T10:48:50
87       010  EINKBELEG  0072097969  0052128438    EKPO  010007209796900020  VRTKZ       U       NaN         2  2018-02-05T00:00:00  1970-01-01T10:48:50
235      010  EINKBELEG  0072099909  0052844578    EKPO  010007209990900030    KEY       I       NaN       NaN  2018-04-11T00:00:00  1970-01-01T09:29:47
...      ...        ...         ...         ...     ...                 ...    ...

## Preprocessing

### Cleaning 'VALUE_NEW':
- Since there are 79 missing value and imputing any values in this case would not make any sense, I will drop them.

In [84]:
df_cdpos = df_cdpos.dropna(subset = ['VALUE_NEW'])
df_cdpos.isna().sum()

MANDANT         0
OBJECTCLAS      0
OBJECTID        0
CHANGENR        0
TABNAME         0
TABKEY          0
FNAME           0
CHNGIND         0
VALUE_NEW       0
VALUE_OLD     179
UDATE           0
UTIME           0
dtype: int64

- Now, I have to carefully drop the rows for 'VALUE_NEW' where the notes are left 

I start by stripping all spaces from that col

In [85]:
# Stripping all spaces from 'VALUE_NEW' column
df_cdpos.loc[:, 'VALUE_NEW'] = df_cdpos['VALUE_NEW'].str.strip()

# Showing the rows that olny contain strings in the column 'VALUE_NEW'
str_values = df_cdpos[df_cdpos['VALUE_NEW'].str.contains('[a-zA-Z]')]
str_values

,MANDANT,OBJECTCLAS,OBJECTID,CHANGENR,TABNAME,TABKEY,FNAME,CHNGIND,VALUE_NEW,VALUE_OLD,UDATE,UTIME
73,010,EINKBELEG,0072097579,0052084474,EKPO,010007209757900010,TXZ01,U,Turnus und Stichprobe für 2018,Zählerwechsel 2018,2018-01-29T00:00:00,1970-01-01T14:30:18
84,010,EINKBELEG,0072097969,0052126271,EKPO,010007209796900010,BEDNR,U,NEHZ18EB,NaN,2018-02-05T00:00:00,1970-01-01T10:35:54
132,010,EINKBELEG,0072097941,0052287101,EKPO,010007209794100010,EREKZ,U,X,NaN,2018-02-20T00:00:00,1970-01-01T13:57:07
139,010,EINKBELEG,0071154204,0052320113,EKPO,010007115420400010,TXZ01,U,PSW - Ölservice Masch. u. GENO 1 u. 2,Ölservice PSW Masch. u. GENO 1 u. 2,2018-02-23T00:00:00,1970-01-01T08:57:46
142,010,EINKBELEG,0071154204,0052320141,EKPO,010007115420400020,TXZ01,U,PSW - IN/OUT-Miete-Reinigung,IN/OUT-Miete-Reinigung,2018-02-23T00:00:00,1970-01-01T09:02:47
...,...,...,...,...,...,...,...,...,...,...,...,...
2463,010,EINKBELEG,4700001474,0077415266,EKPO,010470000147400010,TXZ01,U,V-F-B-S Frischwurstwaren Saunadorf 20-23,V-F-B-S Frischwurstwaren Saunadorf 20-22,2023-01-05T00:00:00,1970-01-01T09:29:08
2464,010,EINKBELEG,4700001196,0077531297,EKPO,010470000119600010,AFNAM,U,VOIGNI001,NaN,2023-01-13T00:00:00,1970-01-01T10:42:57
2469,010,EINKBELEG,4700001402,0077684839,EKPO,010470000140200010,LOEKZ,U,S,NaN,2023-01-24T00:00:00,1970-01-01T12:03:20
2470,010,EINKBELEG,0072119260,0077732552,EKPO,010007211926000010,AFNAM,U,WIECMA001,SPROTI001,2023-01-27T00:00:00,1970-01-01T10:47:50


Let's drop all of them

In [86]:
df_cdpos = df_cdpos[~df_cdpos['VALUE_NEW'].str.contains('[a-zA-Z]')]
df_cdpos[df_cdpos['VALUE_NEW'].str.contains('[a-zA-Z]')].any().sum() # sanity check, should be 0

0

In [87]:
# Let's check the data to see what is going on with these single digits under 'VALUE_NEW'

# Loop to filter for values with length less than 1, 2, and 3
for length_threshold in range(1, 4):
    current_filter = df_cdpos[df_cdpos['VALUE_NEW'].str.len() <= length_threshold]
    print(f"Filter {length_threshold}: {current_filter.shape}")

Filter 1: (39, 12)
Filter 2: (39, 12)
Filter 3: (39, 12)


Good news!

In [88]:
filter_4 = df_cdpos[df_cdpos['VALUE_NEW'].str.len() == 4]
print(filter_4)

     MANDANT OBJECTCLAS    OBJECTID    CHANGENR TABNAME              TABKEY  FNAME CHNGIND VALUE_NEW       VALUE_OLD                UDATE                UTIME
74       010  EINKBELEG  0076052494  0052093316    EKPO  010007605249400010  NETPR       U      3.80            3.40  2018-01-30T00:00:00  1970-01-01T11:13:29
76       010  EINKBELEG  0076052494  0052093327    EKPO  010007605249400010  NETPR       U      4.40            3.80  2018-01-30T00:00:00  1970-01-01T11:13:48
78       010  EINKBELEG  0076052494  0052093318    EKPO  010007605249400010  NETPR       U      3.80            4.40  2018-01-30T00:00:00  1970-01-01T11:14:14
196      010  EINKBELEG  0072095046  0052763151    EKPO  010007209504600010  NETPR       U      1.00         3000.00  2018-04-05T00:00:00  1970-01-01T06:47:15
197      010  EINKBELEG  0072095046  0052763151    EKPO  010007209504600010  NETWR       U      1.00         3000.00  2018-04-05T00:00:00  1970-01-01T06:47:15
203      010  EINKBELEG  0076056219  005276329

Row 481 still looks to be problematic, but we will not need it for the function, since it FNAME is not NETWR in this case. I will just drop it.

Let's convert that into column into a float dtype 

In [89]:
# Drop row 481
df_cdpos.drop(481, inplace=True)

# Convert VALUE_NEW into float
df_cdpos['VALUE_NEW'] = df_cdpos['VALUE_NEW'].astype(float)

Now, let's try to convert 'VALUE_OLD' into float:
- If it is possible, it means that 'VALUE_OLD' no longer has any string representations like 'Zählerwechsel 2018', etc.

In [90]:
df_cdpos['VALUE_OLD'] = df_cdpos['VALUE_OLD'].astype(float)
# It worked, so that's good

### Creating D_DATE (concat UDATE and UTIME)

In [91]:
# Convert 'UDATE' column to datetime data type
df_cdpos['UDATE'] = pd.to_datetime(df_cdpos['UDATE']).dt.strftime('%Y-%m-%d')

# Convert 'UTIME' to datetime and to format 'THH:MM:SS'
df_cdpos['UTIME'] = pd.to_datetime(df_cdpos['UTIME']).dt.strftime('T%H:%M:%S')

# concat UDATE and UTIME into a new column called D_DATE
df_cdpos['D_DATE'] = df_cdpos['UDATE'] + '' + df_cdpos['UTIME']

# Convert D_DATE to datetime
df_cdpos['D_DATE'] = pd.to_datetime(df_cdpos['D_DATE'])

### Let's use feature engineering on TABKEY to get EBELP of the PO_Item

In [92]:
# First let's examine the EBELP column from df_ekpo dataframe
df_ekpo.value_counts('EBELP')

EBELP
00010    207
00020     39
00060     20
00040     12
00050      2
00070      1
00100      1
00110      1
00140      1
dtype: int64

In [93]:
# Creating new column from the 5 last digits and assigning it as 4th column
df_cdpos.insert(3, 'EBELP', df_cdpos['TABKEY'].str[-5:])

# Filtering FNAME
df_cdpos = df_cdpos[df_cdpos['FNAME'] == 'NETWR']

# Dropping unnecessary columns
df_cdpos = df_cdpos.drop(['OBJECTCLAS', 'CHANGENR', 'TABKEY', 'CHNGIND', 'UTIME', 'UDATE', 'TABNAME'], axis=1)

# Renaming columns for the function
df_cdpos = df_cdpos.rename(columns={'MANDANT': 'MANDT', 'OBJECTID': 'EBELN', 'VALUE_NEW': 'NETWR'}).reset_index(drop=True)
df_cdpos.tail(15)

,MANDT,EBELN,EBELP,FNAME,NETWR,VALUE_OLD,D_DATE
1033,010,0071165679,00010,NETWR,1769.92,1671.25,2022-12-20 12:02:11
1034,010,0071167269,00040,NETWR,171.76,17176.00,2022-12-20 12:12:08
1035,010,0072102940,00010,NETWR,7135.50,7023.50,2022-12-21 12:06:50
1036,010,0072109202,00010,NETWR,9000.00,7960.00,2022-12-22 09:01:55
1037,010,0072111995,00010,NETWR,47628.00,100000.00,2022-12-22 11:51:20
1038,010,0072112916,00010,NETWR,7588.10,4000.00,2022-12-23 08:20:55
1039,010,0072112916,00010,NETWR,7587.99,7588.10,2022-12-23 10:39:38
1040,010,4700001545,00010,NETWR,89000.00,49000.00,2023-01-05 09:14:55
1041,010,4700001551,00010,NETWR,16000.00,10000.00,2023-01-05 09:24:33
1042,010,4700001474,00010,NETWR,53000.00,36000.00,2023-01-05 09:29:08


In [94]:
# Print the the earliest order using D_DATE column
print(df_cdpos['D_DATE'].max())

2023-02-08 12:52:58


In [102]:
filter_x = df_cdpos[(df_cdpos['MANDT'] == '010') & (df_cdpos['EBELN'] == '4700001106')]

filter_x.sort_values('D_DATE')

,MANDT,EBELN,EBELP,FNAME,NETWR,VALUE_OLD,D_DATE
0,010,4700001106,00030,NETWR,25000.0,15000.00,2018-01-08 15:54:10
1,010,4700001106,00030,NETWR,50000.0,25000.00,2018-01-09 08:22:23
2,010,4700001106,00030,NETWR,25000.0,50000.00,2018-01-09 09:24:08
8,010,4700001106,00010,NETWR,2500.0,1332.49,2018-01-22 10:10:00
289,010,4700001106,00010,NETWR,4500.0,2500.00,2019-04-12 00:33:36
354,010,4700001106,00010,NETWR,8000.0,4500.00,2019-09-09 10:34:20
545,010,4700001106,00010,NETWR,12000.0,8000.00,2020-06-18 11:21:18


## Now I can start the function that will aggregate PO NETWR sum on a specified date

In [96]:
from datetime import datetime

def calculate_net_order_value(MANDT, EBELN, D_DATE):
    """
    Calculate the net order value for a specific purchase order on a given date.

    Parameters:
        MANDT (str): Unique client ID, e.g. "010".
        EBELN (str): Purchase order number, e.g. "4700001106".
        D_DATE (str): Date in one of the following formats:
            - 'YYYY-MM-DD' for a specific day (e.g., '2018-01-08')
            - 'YYYY-MM' for a specific month (e.g., '2018-01')
            - 'YYYY' for a specific year (e.g., '2020')

    Returns:
        float: Total net order value for the specified purchase order and date.
        pandas.DataFrame: Filtered DataFrame containing the most recent records for each EBELP value.
    """
    filtered_df = df_cdpos[(df_cdpos['MANDT'] == MANDT) & (df_cdpos['EBELN'] == EBELN)]

    if D_DATE:
        try:
            if len(D_DATE) == 10:
                date_value = datetime.strptime(D_DATE, '%Y-%m-%d')
                filtered_df = filtered_df[filtered_df['D_DATE'].dt.date == date_value.date()]
            elif len(D_DATE) == 7:
                date_value = datetime.strptime(D_DATE, '%Y-%m')
                filtered_df = filtered_df[filtered_df['D_DATE'].dt.strftime('%Y-%m') == date_value.strftime('%Y-%m')]
            elif len(D_DATE) == 4:
                date_value = datetime.strptime(D_DATE, '%Y')
                filtered_df = filtered_df[filtered_df['D_DATE'].dt.strftime('%Y') == date_value.strftime('%Y')]
            else:
                raise ValueError
        except ValueError:
            return None

    # Keep only the most recent record for each EBELP
    filtered_df = filtered_df.sort_values(by='D_DATE', ascending=False)
    filtered_df = filtered_df.drop_duplicates(subset='EBELP', keep='first')
    
    net_order_value = filtered_df['NETWR'].sum()

    return net_order_value, filtered_df

In [97]:
# Example usage
MANDT = "010"
EBELN = "4700001106"
D_DATE = "2018-01"

result, filtered_records = calculate_net_order_value(MANDT, EBELN, D_DATE)
if result is not None:
    print(filtered_records)
    print()
    print(f"Total net order value for MANDT {MANDT}, EBELN {EBELN}, and D_DATE {D_DATE}: {result}")
else:
    print("Invalid MANDT/EBELN/D_DATE format")

  MANDT       EBELN  EBELP  FNAME    NETWR  VALUE_OLD              D_DATE
8   010  4700001106  00010  NETWR   2500.0    1332.49 2018-01-22 10:10:00
2   010  4700001106  00030  NETWR  25000.0   50000.00 2018-01-09 09:24:08

Total net order value for MANDT 010, EBELN 4700001106, and D_DATE 2018-01: 27500.0


# Ok so the bulk of the function works. 

Now I have to convert it all into a function to make it generic, so the next time someone loads a different CDPOS file it will work too.